In [8]:
import spacy 
from spacy.matcher import Matcher
from spacy.tokens import Token

nlp = spacy.load("en")
doc = nlp("Test 10 abc")

if not Token.get_extension('mytest'):
    my_test = lambda token: 5
    Token.set_extension('mytest', getter=my_test)

print(my_test)
matcher = Matcher(nlp.vocab)
pattern = [{'SHAPE': 'dd'}, {}]   #  empty dict for "any token", or specify IS_ALPHA etc.
matcher.add('test', None, pattern)

matches = matcher(doc)
for match_id, start, end in matches:
    span = doc[start : end]
    print(span[1]._.mytest)
    # all your matches are two tokens, so you can refer to span[1]
    if span[1]._.mytest == 5:
        print("span" + str(span))

# matcher = Matcher(nlp.vocab)
# pattern = [{'SHAPE': 'dd'}, {}]   #  empty dict for "any token", or specify IS_ALPHA etc.
# matcher.add('test', None, pattern)

# print('a')

# matches = matcher(doc)
# for match_id, start, end in matches:
#     span = doc[start : end]
#     print('aa')
#     # all your matches are two tokens, so you can refer to span[1]
#     if span[1]._.my_test == 2:
#         print(span)

<function <lambda> at 0x10db4bea0>
5
span10 abc


In [2]:
#!python -m spacy validate

In [3]:
import spacy 
from spacy.matcher import Matcher
from spacy.tokens import Token

nlp = spacy.load("en")
#matcher = Matcher(nlp.vocab)

if not Token.get_extension('is_speed'):
    Token.set_extension('is_speed', getter=lambda token: token.text in ('miles', 'mph', 'miles per hour', 'highway speed'))

pattern = [{'LEMMA': 'have'}, {'_': {'is_speed': True}}]
matcher.add('HOW_FAST', None, pattern)

doc = nlp("Car was going 299 highway speed. 809 mph. 88 highway speed")

matches = matcher(doc)

print (matches)
for match_id, start, end in matches:
    span = doc[start : end]
    print(span)
    # all your matches are two tokens, so you can refer to span[1]
    if span[1]._.is_speed:
        print(f'span {span}')

[(1618900948208871284, 10, 12)]
88 highway


In [9]:
from spacy.matcher import PhraseMatcher

matcher = PhraseMatcher(nlp.vocab)
matcher.add('Speed', None, nlp(u'highway speed', 'miles per hour'))
doc = nlp("Car was going 299 highway speed. 809 mph. 88 highway speed")
matches = matcher(doc)

print(matches)

for match_id, start, end in matches:
    span = doc[start : end]
    print(span)

[(17758482761287291853, 4, 6), (17758482761287291853, 11, 13)]
highway speed
highway speed


In [5]:
# https://github.com/explosion/spaCy/issues/1945

from __future__ import unicode_literals
from spacy.lang.en import English
from spacy.matcher import PhraseMatcher

text = "deep machine learning"
mw_list = ["machine learning", "deep blue", "planing machine"]

nlp = English()
matcher = PhraseMatcher(nlp.vocab)
matcher.add("MWE", None, *[nlp.tokenizer(item) for item in mw_list])

print(len(matcher(nlp(text))) == 1)
print(matcher(nlp(text)))

True
[(11792181371108107841, 1, 3)]


In [6]:
# import spacy
# from spacy.matcher import Matcher

# nlp = spacy.load("en")
# matcher = Matcher(nlp.vocab)
# #pattern = [{'_': {'speed': {'REGEX': '[\d{2}]'}}}]
# #pattern = [{'_': {'country': {'REGEX': '^([Uu](\\.?|nited) ?[Ss](\\.?|tates)'}}}]
# #pattern = [{'LEMMA': {'REGEX': '^([Uu](\\.?|nited) ?[Ss](\\.?|tates)'}}, {'LOWER': 'president'}]
                 
# matcher.add('SPEED', None, pattern)
# doc = nlp("Car was going 55 highway")
# matches = matcher(doc)

# print (matches)
    
# for match_id, start, end in matches:
#     span = doc[start : end]
#     print(span)
     


[]


In [22]:
matcher = Matcher(nlp.vocab)
#pattern = [{'SHAPE': 'dd'}, {'IS_DIGIT': True}]   #  empty dict for "any token", or specify IS_ALPHA etc.
pattern = [{"IS_DIGIT": True}]   #  empty dict for "any token", or specify IS_ALPHA etc.
matcher.add('test', None, pattern)

#doc = nlp("Car was going 55 highway")
doc = nlp("55 highway")
matches = matcher(doc)

print(matches)

for match_id, start, end in matches:
    span = doc[start : end]
    # all your matches are two tokens, so you can refer to span[1]
    print("span" + str(span))

[(1618900948208871284, 0, 1)]
span55


In [72]:
# https://spacy.io/usage/linguistic-features#regex
import spacy
from spacy.matcher import Matcher
import re

nlp = spacy.load('en_core_web_sm')
speed_flag = lambda text: bool(re.search(r'(?i)\d+\s?mph', text))
IS_SPEED = nlp.vocab.add_flag(speed_flag)

matcher = Matcher(nlp.vocab)
matcher.add('MPH', None, [{IS_SPEED: True}])

doc = nlp(u'44 MPH hhh.')
matches = matcher(doc)

print(matches)
for match_id, start, end in matches:
    span = doc[start:end]
    print(span.text)

[(14771884224424469583, 0, 1)]
444


In [73]:
nlp = spacy.load('en_core_web_sm')
speed_flag = lambda text: bool(re.search(r'(?i)s?mph', text))
IS_USPEED = nlp.vocab.add_flag(speed_flag)

matcher = Matcher(nlp.vocab)
matcher.add('MPH', None, [{IS_USPEED: True}])

doc = nlp(u'444 MPH hhh.')
matches = matcher(doc)

print(matches)
for match_id, start, end in matches:
    span = doc[start:end]
    print(span.text)

[(14771884224424469583, 1, 2)]
MPH


In [81]:
# https://github.com/explosion/spaCy/issues/882
# https://github.com/explosion/spaCy/issues/1567#issuecomment-356589346
import spacy
import re
nlp = spacy.load('en', disable=['parser', 'tagger', 'ner'])
doc = nlp("This is a number: 5634 mph. This is another number: 90 mpH.")

NUM_PATTERN = re.compile(r"(?i)\d+\s?mph")

for match in re.finditer(NUM_PATTERN, doc.text):
    start, end = match.span()
    print(f"The matched text: '{doc.text[start:end]}'")
    span = doc.char_span(start, end)
    # Now you have a Span object and you can do everything.

The matched text: '5634 mph'
The matched text: '90 mpH'


In [96]:


import spacy
import re
nlp = spacy.load('en', disable=['parser', 'tagger', 'ner'])
doc = nlp("This is a number: 5634 miles perh. This is another number: 90 m pH.")

#NUM_PATTERN = re.compile(r"(?i)\d+\s?mph") #r'deff?in[ia]tely'
NUM_PATTERN = re.compile(r'(?i)\d+\s?m(iles)?\s?p(er)?\s?h(our)?')

for match in re.finditer(NUM_PATTERN, doc.text):
    start, end = match.span()
    print(f"The matched text: '{doc.text[start:end]}'")
    span = doc.char_span(start, end)
    # Now you have a Span object and you can do everything.

The matched text: '5634 miles perh'
The matched text: '90 m pH'
